In [102]:
import pandas as pd
import ast

data = pd.read_csv("/content/challenge3.csv")
ack = pd.read_csv("/content/ack_log.csv")
filtered_pubs = pd.read_csv("/content/filtered_pubs.csv")
id_log = pd.read_csv("/content/id_log.csv")

In [74]:
data

,No.,Time,Source,Destination,Protocol,Length,Source Port,Destination Port,Info,Payload
0,1,1.631806,10.0.2.15,3.65.137.17,MQTT,79,34039,1883,Connect Command,NaN
1,2,1.633054,10.0.2.15,3.65.137.17,MQTT,85,47723,1883,Connect Command,NaN
2,3,1.656703,3.65.137.17,10.0.2.15,MQTT,62,1883,34039,Connect Ack,NaN
3,4,1.666551,10.0.2.15,91.121.93.94,MQTT,80,43133,1883,Connect Command,NaN
4,5,1.673415,3.65.137.17,10.0.2.15,MQTT,67,1883,47723,Connect Ack,NaN
...,...,...,...,...,...,...,...,...,...,...
7706,7707,2397.304498,52.29.173.150,10.0.2.15,MQTT,62,1883,55677,Ping Response,NaN
7707,7708,2401.957147,10.0.2.15,52.29.173.150,MQTT,58,60609,1883,Ping Request,NaN
7708,7709,2401.972165,52.29.173.150,10.0.2.15,MQTT,62,1883,60609,Ping Response,NaN
7709,7710,2406.326562,10.0.2.15,3.65.137.17,MQTT,58,34039,1883,Ping Request,NaN


In [5]:
ack

,timestamp,sub_id,msg_type
0,1713251034898,455,Connect Ack
1,1713251259979,590,Publish Ack (id=11)


In [6]:
filtered_pubs

,no,long,range,lat,type,unit,description
0,0,49,"[7,35]",84,temperature,F,Room Temperature
1,1,75,"[6,41]",84,temperature,F,Room Temperature
2,2,90,"[1,49]",52,temperature,F,Room Temperature
3,3,60,"[7,30]",93,temperature,F,Room Temperature
4,4,92,"[0,53]",64,temperature,F,Room Temperature
5,5,90,"[1,49]",52,temperature,F,Room Temperature
6,6,100,"[9,30]",75,temperature,F,Room Temperature
7,7,81,"[3,31]",99,temperature,F,Room Temperature
8,8,61,"[8,45]",77,temperature,F,Room Temperature


In [7]:
id_log

,no,id,timestamp
0,0,1097.932978,1713250989102
1,1,392.049684,1713250994107
2,2,2587.994134,1713250999109
3,3,3076.811463,1713251004111
4,4,975.278711,1713251009113
...,...,...,...
76,76,2941.367189,1713251369253
77,77,1414.871478,1713251374255
78,78,1345.935226,1713251379256
79,79,3562.695223,1713251384257


In [168]:
def check_ack(sub_id):
  if not sub_id in ack_list:
    raise Exception("missing ack for id:", sub_id)

def check_pub(payload, id):
  i=0
  tmp=False
  for p in payload:
    if p.get("unit") == "F" and p.get("type") ==  "temperature":
      for i in range(0, len(filtered_pubs)):
        if p.get("long") == filtered_pubs["long"][i] and p.get("lat") == filtered_pubs["lat"][i] and p.get("description") == filtered_pubs["description"][i]:
          tmp=True
          break
      #print(id, i, p)
      if(tmp==False): raise Exception("expected tupla from message with id %d in filtered_pubs" %id)
  return i

In [169]:
ack_list = ack["sub_id"].to_list()
ack_n = 0

pubs_n = 0


for i in range(1, len(id_log) - 1):
  id = int(id_log["id"][i] % 7711)

  for j in range(1, len(data) -1):
    if  data["No."][j] == id and "Ack" in data["Info"][j]:
      check_ack(id)
      ack_n +=1
    elif data["No."][j] == id and "Publish Message" in data["Info"][j]:
      if( not pd.isna(data["Payload"][j])):
        pubs_n += check_pub(ast.literal_eval("["+data["Payload"][j]+"]"), id)


if len(ack_list) != ack_n:
  raise Exception("you have %d ack message saved but expected ack numbers is" % ack_n, len(ack_list))


if len(filtered_pubs) != pubs_n:
  raise Exception("you have %d pub message saved but expected pub numbers is" % len(filtered_pubs), pubs_n)

Exception: expected tupla from message with id 64 in filtered_pubs